In [1]:
import sys
from pathlib import Path
import os 
sys.path.append('/home/gridsan/vyuan/.local/lib/python3.9/site-packages/')

current_path = Path.cwd()

# Go to top of the root and append
root = current_path.parents[3]
sys.path.append(str(root))

os.chdir(root)

In [2]:
import json

from fairness_cv_project.methods.label_free_cbm.src.utils import data_utils
from fairness_cv_project.methods.label_free_cbm.src.utils import conceptset_utils

In [3]:
"""
CLASS_SIM_CUTOFF: Concenpts with cos similarity higher than this to any class will be removed
OTHER_SIM_CUTOFF: Concenpts with cos similarity higher than this to another concept will be removed
MAX_LEN: max number of characters in a concept

PRINT_PROB: what percentage of filtered concepts will be printed
"""

CLASS_SIM_CUTOFF = 0.85
OTHER_SIM_CUTOFF = 0.9
MAX_LEN = 30
PRINT_PROB = 1

dataset = "phoning_eating"
device = "cuda"

In [8]:
# CASE 1: You just generated concepts from GPT
# EDIT these to use the initial concept sets you want

save_name = "data/concept_sets/{}_filtered_new.txt".format(dataset)

with open(root / "data/concept_sets/gpt3_init/gpt3_{}_important_new.json".format(dataset), "r") as f:
    important_dict = json.load(f)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_superclass_new.json".format(dataset), "r") as f:
    superclass_dict = json.load(f)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_around_new.json".format(dataset), "r") as f:
    around_dict = json.load(f)
# with open(root / "data/concept_sets/gpt3_init/gpt3_{}_job_new.json".format(dataset), "r") as f:
#     job_dict = json.load(f)
    
with open(root / data_utils.LABEL_FILES[dataset], "r") as f:
    classes = f.read().split("\n")
    
concepts = set()

for values in important_dict.values():
    concepts.update(set(values))

for values in superclass_dict.values():
    concepts.update(set(values))
    
for values in around_dict.values():
    concepts.update(set(values))
    
# for values in job_dict.values():
#    concepts.update(set(values))

print(len(concepts))

45


In [10]:
# CASE 2: Read another concept set, and filter it

save_name = "data/concept_sets/{}_augmented_filtered.txt".format(dataset)

with open(root / 'data' / 'concept_sets' / 'phoning_eating_augmented.txt') as f:
    concepts = set(f.read().split("\n"))
    
with open(root / data_utils.LABEL_FILES[dataset], "r") as f:
    classes = f.read().split("\n")
    

In [11]:
concepts = conceptset_utils.remove_too_long(concepts, MAX_LEN, PRINT_PROB)

66 66


In [12]:
concepts = conceptset_utils.filter_too_similar_to_cls(concepts, classes, CLASS_SIM_CUTOFF, device, PRINT_PROB)

66
66
Class:eating - Concept:food, sim:0.882 - Deleting food

Class:phoning - Concept:phone, sim:0.856 - Deleting phone

64


In [13]:
concepts = conceptset_utils.filter_too_similar(concepts, OTHER_SIM_CUTOFF, device, PRINT_PROB)

a fork - fork , sim:0.9325 - Deleting fork
a kitchen - kitchen , sim:0.9475 - Deleting kitchen
a plate - plate , sim:0.9435 - Deleting plate
a spoon - spoon , sim:0.9400 - Deleting spoon
a table - table , sim:0.9492 - Deleting table
outdoors - outside , sim:0.9430 - Deleting outdoors
58


In [14]:
with open(save_name, "w") as f:
    f.write(concepts[0])
    for concept in concepts[1:]:
        f.write("\n" + concept)